In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import torch
from backtesting import Strategy

# Import your GRU model (assumes PyTorch Lightning or plain PyTorch model)
from models.gru_model import GRUModule

C:\Users\yoyo\miniconda3\envs\fxml\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
# Load the GRU model once globally (not in the strategy)
CHECKPOINT_PATH = '../lightning_logs/prob_gru/version_9/checkpoints/best_checkpoint.ckpt'
model = GRUModule.load_from_checkpoint(CHECKPOINT_PATH)
model.eval()
model.to('cpu')  # or 'cuda' if you're running with GPU

GRUModule(
  (model): GRUModel(
    (gru): GRU(1, 256, num_layers=3, batch_first=True, dropout=0.8)
    (linear): Linear(in_features=256, out_features=3, bias=True)
    (softmax): Softmax(dim=1)
  )
  (criterion): CrossEntropyLoss()
)

In [18]:
class GRUStrategy(Strategy):
    def init(self):
        self.sequence_length = 30
        self.lot_size = 0.1
        self.stop_loss_amount = 0.05
        self.take_profit_amount = 0.075

        self.buy_count = 0
        self.sell_count = 0

    def next(self):
        if len(self.data.Close) < self.sequence_length + 1:
            return  # Not enough data to predict

        # Extract the last N closes
        close_prices = self.data.Close[-self.sequence_length:]
        close_returns = pd.Series(close_prices).pct_change().dropna().values.reshape(1, -1, 1)
        close_tensor = torch.FloatTensor(close_returns)

        # Model prediction
        with torch.no_grad():
            _, probs = model(close_tensor)
            signal = torch.argmax(probs, dim=1).item()
            if probs[0, signal] < 0.7:
                return
        current_price = self.data.Close[-1]

        # 0 = sell, 1 = hold, 2 = buy
        if signal == 2:
            sl = current_price - self.stop_loss_amount
            tp = current_price + self.take_profit_amount
            self.buy(size=self.lot_size, sl=sl, tp=tp)
            self.buy_count += 1

        if signal == 0:
            sl = current_price + self.stop_loss_amount
            tp = current_price - self.take_profit_amount
            self.sell(size=self.lot_size, sl=sl, tp=tp)
            self.sell_count += 1

In [36]:
PKL_PATH = '../data/processed/usdjpy-bar-2025-01-01-2025-05-12_processed.pkl'

df = pd.read_pickle(PKL_PATH)
# df = df[df['timestamp'].dt.year == 2024].copy()
df.head()

,timestamp,open,high,low,close,volume,time_group,close_delta,close_return,close_direction,prob_down,prob_flat,prob_up,label
9,2025-01-01 22:15:00,157.2400,157.24,157.2370,157.2375,12.000000,2,0.0000,0.000000,flat,0.0,1.0,0.0,1
10,2025-01-01 22:16:00,157.2400,157.24,157.2375,157.2400,32.400001,2,0.0025,0.000016,flat,0.0,1.0,0.0,1
11,2025-01-01 22:17:00,157.2390,157.24,157.2375,157.2400,14.400001,2,0.0000,0.000000,flat,0.0,1.0,0.0,1
12,2025-01-01 22:18:00,157.2375,157.24,157.2375,157.2375,6.600000,2,-0.0025,-0.000016,flat,0.0,1.0,0.0,1
13,2025-01-01 22:19:00,157.2400,157.24,157.2375,157.2395,19.200001,2,0.0020,0.000013,flat,0.0,1.0,0.0,1


In [37]:
from backtesting import Backtest

In [38]:
# Format the DataFrame for backtesting.py
df['time'] = pd.to_datetime(df['timestamp'])

df.set_index('time', inplace=True)
df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)

print(df.head())

# Run backtest
test = Backtest(df, GRUStrategy, cash=10000, hedging=True, exclusive_orders=True)
result = test.run()

print(result)
print(f'Buy count = {result._strategy.buy_count}')
print(f'Sell count = {result._strategy.sell_count}')

                              timestamp      Open    High       Low     Close  \
time                                                                            
2025-01-01 22:15:00 2025-01-01 22:15:00  157.2400  157.24  157.2370  157.2375   
2025-01-01 22:16:00 2025-01-01 22:16:00  157.2400  157.24  157.2375  157.2400   
2025-01-01 22:17:00 2025-01-01 22:17:00  157.2390  157.24  157.2375  157.2400   
2025-01-01 22:18:00 2025-01-01 22:18:00  157.2375  157.24  157.2375  157.2375   
2025-01-01 22:19:00 2025-01-01 22:19:00  157.2400  157.24  157.2375  157.2395   

                        Volume  time_group  close_delta  close_return  \
time                                                                    
2025-01-01 22:15:00  12.000000           2       0.0000      0.000000   
2025-01-01 22:16:00  32.400001           2       0.0025      0.000016   
2025-01-01 22:17:00  14.400001           2       0.0000      0.000000   
2025-01-01 22:18:00   6.600000           2      -0.0025     -0.0000

Backtest.run:   0%|          | 0/129817 [00:00<?, ?bar/s]

Start                     2025-01-01 22:15:00
End                       2025-05-11 23:59:00
Duration                    130 days 01:44:00
Exposure Time [%]                    99.84979
Equity Final [$]                    10079.417
Equity Peak [$]                     10079.534
Return [%]                            0.79417
Buy & Hold Return [%]                -7.24763
Return (Ann.) [%]                     1.76677
Volatility (Ann.) [%]                 0.92885
CAGR [%]                              1.54434
Sharpe Ratio                          1.90209
Sortino Ratio                         3.05242
Calmar Ratio                           3.1382
Alpha [%]                             0.80242
Beta                                  0.00114
Max. Drawdown [%]                    -0.56299
Avg. Drawdown [%]                    -0.01113
Max. Drawdown Duration       37 days 03:29:00
Avg. Drawdown Duration        0 days 09:09:00
# Trades                               102584
Win Rate [%]                      

In [39]:
test.plot()

C:\Users\yoyo\miniconda3\envs\fxml\Lib\site-packages\backtesting\_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '30min'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p2892', ...)

In [32]:
test.plot(
    results=None,
    filename=None,
    plot_width=None,
    plot_equity=True,
    plot_return=False,
    plot_pl=False,
    plot_volume=False,
    plot_drawdown=False,
    plot_trades=False,
    smooth_equity=False,
    relative_equity=True,
    superimpose=False,
    resample=True,
    reverse_indicators=False,
    show_legend=False,
    open_browser=False
)

C:\Users\yoyo\miniconda3\envs\fxml\Lib\site-packages\backtesting\_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '1h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p2550', ...)